# wavy woven vase

This notebook generates a 3D visualization of a wavy woven vase with a solid base.

*<<< check out other demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*

run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
# vase parameters
params = dict(height=120, base_height=5, base_radius=25, top_radius=12,
              amplitude=4, angular_waves=8, vertical_waves=4,
              n_theta=200, n_z=200)


In [ ]:
def generate_vase_surface(height=120, base_height=5, base_radius=25, top_radius=12,
                            amplitude=4, angular_waves=8, vertical_waves=4,
                            n_theta=200, n_z=200):
    """Return X, Y, Z coordinate arrays for a woven vase."""
    theta = np.linspace(0, 2*np.pi, n_theta)
    z = np.linspace(0, height, n_z)
    Theta, Z = np.meshgrid(theta, z)
    u = np.clip((Z - base_height) / (height - base_height), 0, 1)
    base_section = Z < base_height
    radius = base_radius + (top_radius - base_radius) * u
    wave = amplitude * (1 - u) * np.sin(angular_waves * Theta) * np.sin(vertical_waves * np.pi * u)
    R = np.where(base_section, base_radius, radius + wave)
    X = R * np.cos(Theta)
    Y = R * np.sin(Theta)
    return X, Y, Z

In [ ]:
def plot_vase(**kwargs):
    X, Y, Z = generate_vase_surface(**kwargs)
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', showscale=False)])
    fig.update_layout(scene_aspectmode='data', title='Wavy Woven Vase', margin=dict(l=0, r=0, b=0, t=40))
    fig.show()
    return X, Y, Z

In [ ]:
# visualize with default parameters
X, Y, Z = plot_vase(**params)

In [ ]:
def save_as_stl(X, Y, Z, path='wavy_woven_vase.stl'):
    n_z, n_theta = X.shape
    vertices = np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))
    faces = []
    for i in range(n_z-1):
        for j in range(n_theta):
            jp = (j+1) % n_theta
            a = i*n_theta + j
            b = (i+1)*n_theta + j
            c = (i+1)*n_theta + jp
            d = i*n_theta + jp
            faces.append([a, b, c])
            faces.append([a, c, d])
    center = len(vertices)
    vertices = np.vstack([vertices, [0, 0, 0]])
    for j in range(n_theta):
        jp = (j+1) % n_theta
        faces.append([center, jp, j])
    with open(path, 'w') as f:
        f.write('solid vase
')
        for tri in faces:
            p0, p1, p2 = vertices[tri]
            n = np.cross(p1-p0, p2-p0); n /= (np.linalg.norm(n) + 1e-8)
            f.write(f' facet normal {n[0]:e} {n[1]:e} {n[2]:e}
')
            f.write('  outer loop
')
            for v in (p0, p1, p2):
                f.write(f'   vertex {v[0]:e} {v[1]:e} {v[2]:e}
')
            f.write('  endloop
 endfacet
')
        f.write('endsolid vase
')

In [ ]:
# uncomment to save the mesh as an STL file
# save_as_stl(X, Y, Z)